In [1]:
rawinput = 'ljoxqyyw'

In [2]:
import numpy as np

In [3]:
# Part 1
def knot_hash(in_str):
    lengths = [ord(i) for i in in_str] + [17, 31, 73, 47, 23]
    n_nodes = 256
    nodes = np.arange(n_nodes)
    current_pos = 0
    skipsize = 0
    for i in range(64):
        for length in lengths:
            ix = np.mod(np.arange(length) + current_pos, n_nodes)
            nodes[ix] = nodes[ix[::-1]]
            current_pos = (current_pos+length+skipsize) % n_nodes
            skipsize += 1

    xor_all = lambda x: (np.squeeze(x,1) 
                         if x.shape[1] == 1 
                         else xor_all(np.column_stack((x[:, 0] ^ x[:, 1], x[:, 2:]))))

    return ''.join(np.vectorize(lambda x: f'00{hex(x)[2:]}'[-2:])(xor_all(nodes.reshape(16,16))))

cells = (np.vectorize(lambda x: f'0000{bin(int(x, 16))[2:]}'[-4:])
         (np.array([knot_hash(f'{rawinput}-{i}') for i in range(128)])
          .view('<U1').reshape(-1, 32))
         .view('<U1'))

np.sum(cells == '1').item()

8316

In [4]:
# Part 2
mergeiter = lambda x,y=4: mergeiter(np.append(x[:1], np.where((z:=np.minimum(x[1:], x[:-1]))<0, x[1:], z), axis=0)[::-1].T, y-1) if y else x

gps = np.where((z:=cells.reshape(-1))=='1', np.arange(z.shape[0]), -1).reshape(cells.shape)
while np.any(gps != (gps:=mergeiter(gps))):  pass

np.unique(gps).shape[0]-1

1074